In [31]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy.optimize import minimize

while True:
    try:
        num_stocks = int(input("Please enter the number of stocks you want to input: "))
        if num_stocks <= 0:
            print("The number of stocks must be a positive integer. Please try again.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a valid integer.")

stock_symbols = []

for i in range(num_stocks):
    while True:
        stock_symbol = input(f"Enter stock symbol {i + 1} (e.g., AAPL): ")
        if stock_symbol.isalpha():  
            stock_symbols.append(stock_symbol.upper())
            break
        else:
            print("Invalid stock symbol. Please enter only letters.")


# Fetch data using yfinance

Yf_ticker_input = " ".join(stock_symbols).lower()
Stock = yf.Tickers(Yf_ticker_input)
df = Stock.history(period="max")

## As per usual we are going to want to calculate the log returns of each column.
Closing_prices = df["Close"]
log_return = np.log(Closing_prices/Closing_prices.shift(1)).dropna()

## We then want to calculate the covariance matrix in between all stocks to gain a better understanding of the relationship between
## their movements.

covariance_matrix = log_return.cov()

## We then want to calculate the standard deviation of the porfolio itself through matrix multuiplication of the weights of each stock
def standard_deviation(weights, covariance):
    variance = weights.T @ covariance @ weights
    return np.sqrt(variance)

## We know want to predict the future returns of the stocks in the portfolio, we will use the assumption that they follow past historical data.
def predicted_returns(df, weights):
    return np.sum(log_return.mean()*weights)*252

## Here we calculate the sharpe ratio from before, but this time based off multiple stocks

def sharpe_ratio(weights, log_returns, cov_matrix):
    return (predicted_returns(df, weights) - 0.02) / standard_deviation(weights,cov_matrix)

## Remember From CO 250 that we can minimize a problem by maximizing its negative. So we create a function that takes the negative of the sharpe ratio.

def negative_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix)

## We set the bounds of the optimization problem
constraints = {"type" : "eq" , "fun": lambda weights: np.sum(weights) - 1}
initial_weights = np.array([1/len(stock_symbols)] * len(stock_symbols))

# Extract the optimal weights
optimal_weights = optimized_results.x

# Display the optimal weights
print("Optimal Weights:")
for ticker, weight in zip(stock_symbols, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

# Calculate and display the optimal portfolio's metrics
optimal_portfolio_return = predicted_returns(df, optimal_weights)
optimal_portfolio_volatility = standard_deviation(optimal_weights, covariance_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_return, covariance_matrix)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")




[*********************100%%**********************]  3 of 3 completed

Optimal Weights:
SOXL: 0.5000
AAPL: 0.5000
ZM: 0.0000
Expected Annual Return: 0.2450
Expected Volatility: 0.0414
Sharpe Ratio: 5.4401
